In [20]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

xy = np.loadtxt('C:\\Users\\korea\\Desktop\\test.csv', delimiter=',', dtype=np.float32)
x_data=xy[:,0:-1]
y_data=xy[:,[-1]]
F_data=xy[:,:]

# Model Inputs
def model_inputs(real_dim, noise_dim):
    inputs_real_ = tf.placeholder(tf.float32, shape=[None, real_dim], name='inputs_real')
    inputs_z_ = tf.placeholder(tf.float32, shape=[None, noise_dim], name='inputs_z')
    
    return inputs_real_, inputs_z_

def leaky_relu(x, alpha):
    return tf.maximum(alpha * x, x)

# Generator Network# Genera 
def model_generator(z_input, out_dim, n_units=128, reuse=False, alpha=0.01):
    # used to reuse variables, name scope
    with tf.variable_scope('generator', reuse=reuse):
        hidden_layer = tf.layers.dense(z_input, n_units, activation=tf.nn.relu)
        
        logits = tf.layers.dense(hidden_layer, out_dim, activation=None)
        outputs = tf.nn.tanh(logits)
        
        return outputs, logits

# Discriminator Network
def model_discriminator(input, n_units=128, reuse=False, alpha=0.01):
    with tf.variable_scope('discriminator', reuse=reuse):
        hidden_layer = tf.layers.dense(input, n_units, activation=None)
        hidden_layer = leaky_relu(hidden_layer, alpha)
        
        logits = tf.layers.dense(hidden_layer, 1, activation=None)
        outputs = tf.nn.sigmoid(logits)
        
        return outputs, logits



In [24]:
input_size = 39
z_dim = 21
g_hidden_size = 128
d_hidden_size = 128
alpha = 0.01
smooth = 0.1
learning_rate = 0.01

tf.reset_default_graph()  # If we don't have this, as we call this block over and over, the graph gets bigger and bigger

graph = tf.Graph()
with graph.as_default():
    inputs_real, inputs_z = model_inputs(input_size, z_dim)
    
    g_outputs, g_logits = model_generator(inputs_z, input_size, n_units=g_hidden_size, reuse=False, alpha=alpha)
    
    d_outputs_real, d_logits_real = model_discriminator(inputs_real, n_units=d_hidden_size, reuse=False, alpha=alpha)
    d_outputs_fake, d_logits_fake = model_discriminator(g_outputs, n_units=d_hidden_size, reuse=True, alpha=alpha)
    
    d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_real, labels=tf.ones_like(d_logits_real) * (1-smooth)))
    d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake, labels=tf.zeros_like(d_logits_fake)))
    
    d_loss = d_loss_real + d_loss_fake
    g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake, labels=tf.ones_like(d_logits_fake)))
    
    t_vars = tf.trainable_variables()
    g_vars = [variable for variable in t_vars if 'generator' in variable.name]
    d_vars = [variable for variable in t_vars if 'discriminator' in variable.name]
    
    # Affected Variables with var_list
    d_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(d_loss, var_list=d_vars)
    g_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(g_loss, var_list=g_vars)
    
    # Saving variables with var_list
    saver = tf.train.Saver(var_list=g_vars)

In [25]:
samples = []
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(500):
        
        batch_images = F_data[step].reshape([1, 39])
        batch_z = np.random.uniform(-1, 1, size=[1, z_dim])
        
        _ = sess.run(d_optimizer, feed_dict={inputs_real : batch_images, inputs_z : batch_z})
        _ = sess.run(g_optimizer, feed_dict={inputs_z : batch_z})
        loss_d, loss_g = sess.run([d_loss, g_loss], feed_dict={inputs_real : batch_images, inputs_z : batch_z})
        if step%100==0:
            print('step {} / {} Complete. D_Loss : {:0.3f}, G_Loss : {:0.3f}'.format(step+1, 10000, loss_d, loss_g))
        
        sample_z = np.random.uniform(-1, 1, size=[1, z_dim])  # 16 Samples each epoch
        gen_samples, _ = sess.run(model_generator(inputs_z, input_size, reuse=True), feed_dict={inputs_z : sample_z})

        samples.append(gen_samples)

    print('Training Complete. ')

step 1 / 10000 Complete. D_Loss : 2.885, G_Loss : 0.649
step 101 / 10000 Complete. D_Loss : 23.903, G_Loss : 7.514
step 201 / 10000 Complete. D_Loss : 13.317, G_Loss : 3.086
step 301 / 10000 Complete. D_Loss : 22.142, G_Loss : 4.670
step 401 / 10000 Complete. D_Loss : 24.342, G_Loss : 4.071
Training Complete. 


In [72]:
#xy = np.loadtxt('C:\\Users\\SANHA\\Desktop\\test.csv', delimiter=',', dtype=np.float32)
#x_data=xy[:,0:-1]
#y_data=xy[:,[-1]]
#F_data=xy[:,:]

x1_data=gen_samples[:,0:-1]
y1_data=gen_samples[:,[-1]]



nb_classes=2

X=tf.placeholder(tf.float32,[None,38])
Y=tf.placeholder(tf.int32,[None,1])

Y_one_hot=tf.one_hot(Y,nb_classes)
Y_one_hot=tf.reshape(Y_one_hot,[-1,nb_classes])

W=tf.Variable(tf.random_normal([38,nb_classes]),name='weight')
b=tf.Variable(tf.random_normal([nb_classes]),name='bias')

logits=tf.matmul(X,W)+b
hypothesis=tf.nn.softmax(logits)

cost_i=tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=Y_one_hot)

cost=tf.reduce_mean(cost_i)
optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction=tf.argmax(hypothesis,1) #가능성을 퍼센트로~~
correct_prediction=tf.equal(prediction,tf.arg_max(Y_one_hot,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(1000):
        sess.run(optimizer,feed_dict={X:x1_data,Y:y1_data})
        if step %100==0:
            loss,acc=sess.run([cost,accuracy],feed_dict={X:x_data,Y:y_data})
            print("step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step,loss,acc))
 
    print("training by gan sample")
  
    tr=0
    fa=0
    total=0


    pred = sess.run(prediction, feed_dict={X: x_data})
    for p, y in zip(pred, y_data.flatten()):
            if(p==int(y)):
                tr=tr+1
            else:
                fa=fa+1
            #print("[{}] Prediction: {} Real Y: {}".format(p == int(y), p, int(y)))

    print("true={} false: {} acc: {}".format(tr,fa,tr/(tr+fa)))

step:     0	Loss: 34199.516	Acc: 80.05%
step:   100	Loss: 34439.152	Acc: 80.04%
step:   200	Loss: 34601.102	Acc: 80.04%
step:   300	Loss: 34723.578	Acc: 80.05%
step:   400	Loss: 34822.129	Acc: 80.05%
step:   500	Loss: 34904.586	Acc: 80.05%
step:   600	Loss: 34975.441	Acc: 80.06%
step:   700	Loss: 35037.629	Acc: 80.06%
step:   800	Loss: 35093.016	Acc: 80.05%
step:   900	Loss: 35142.949	Acc: 80.05%
training by gan sample
true=20163 false: 5029 acc: 0.8003731343283582


array([4, 2, 0, ..., 0, 0, 0], dtype=int64)